In [52]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import random
import cv2
from matplotlib import pyplot as plt
from shapely.geometry import Polygon

In [41]:
dataset_path = "../data/processed/image-deduplication_hhf"

colors = {0: (0,255,255), 1: (0,255,0), 2: (255,0,255)}

# Подготовка данных для детекции head, face

In [131]:
def yolo2bb(box, dw, dh):
    class_id, x_center, y_center, w, h = box.strip().split()
    x_center, y_center, w, h = float(x_center), float(y_center), float(w), float(h)
    x_center = round(x_center * dw)
    y_center = round(y_center * dh)
    w = round(w * dw)
    h = round(h * dh)
    x = round(x_center - w / 2)
    y = round(y_center - h / 2)

    return class_id, x, y, x+w, y+h

def crop_yolo(x_center, y_center, w, h, image):
    x_center, y_center, w, h = float(x_center), float(y_center), float(w), float(h)
    x_center = round(x_center * dw)
    y_center = round(y_center * dh)
    w = round(w * dw)
    h = round(h * dh)
    x = round(x_center - w / 2)
    y = round(y_center - h / 2)

    imgCrop = image[y:y + h, x:x + w]
    return imgCrop

def draw_label(class_id, box, image):
    
    x_l, y_t, x_r, y_b = box

    # Line thickness of -1 px
    # Thickness of -1 will fill the entire shape
    thickness = 1
    
    image = cv2.rectangle(image, (x_l, y_t), (x_r, y_b), colors[int(class_id)], thickness)
    plt.imshow(image)
    plt.show()

def bb_to_yolo(box, image_w, image_h):
    x1, y1, x2, y2 = box
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]

def increase_bb(box, p, h, w):
    
    _, xmin, ymin, xmax, ymax = box
    
    xmin = max(0, xmin - p * (xmax - xmin))
    xmax = min(w, xmax + p * (xmax - xmin))
    ymin = max(0, ymin - p * (ymax - ymin))
    ymax = min(h, ymax + p * (ymax - ymin))
    
    return int(xmin), int(ymin), int(xmax), int(ymax)

In [141]:
path_label = dataset_path + '/labels/val/'
path_images = dataset_path + '/images/val/'

path_res_labels = dataset_path + '/labels/res_val/'
path_res_images = dataset_path + '/images/res_val/'

for label_file in os.listdir(path_label):
    
    print(label_file)
    
    base_name = os.path.splitext(label_file)[0]
    image = cv2.imread(path_images + base_name + '.jpg')
    
    dh, dw, _ = image.shape
    
    with open(os.path.join(path, label_file), 'r') as label_data:
        bboxes = []
        for box in label_data:
            bboxes.append(yolo2bb(box, dw, dh))
        
        i = 0
        
        for box in bboxes:
            
            class_id, x_l, y_t, x_r, y_b = box
            
            if class_id == '2':
                
                x_l, y_t, x_r, y_b = increase_bb(box, 0.15, dh, dw)
                
                imgCrop = image[y_t:y_b, x_l:x_r]
                
                cv2.imwrite(path_res_images + base_name + '_' + str(i) + '.jpg', imgCrop)
                
                
                dhc, dwc, _ = imgCrop.shape
                
                
                # create label file
                with open(path_res_labels + base_name + '_' + str(i) + '.txt', 'w') as f:
                
                    
                    i += 1
                
                    for box in bboxes:

                        class_id, bx_l, by_t, bx_r, by_b = box

                        if class_id != '2' and ( class_id == '0' or class_id == '1' ):

                            polya = Polygon([(x_l, y_t), (x_l, y_b), (x_r, y_b), (x_r, y_t)]) 
                            polyb = Polygon([(bx_l, by_t), (bx_l, by_b), (bx_r, by_b), (bx_r, by_t)]) 

                            if polya.contains(polyb):
                                new_bb = (bx_l-x_l, by_t-y_t, bx_r-x_l, by_b-y_t)
                                
#                                 draw_label(class_id, new_bb, imgCrop)

                                lbl = bb_to_yolo(new_bb, dwc, dhc)
                                
                                data = class_id + ' ' + ' '.join(str(e) for e in lbl) + '\n'
                                
                                f.write(data)

23-11-2021_02-37-54_PM.txt
